In [81]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from pckgs.helper import timeseries_to_supervised2
from pckgs.models import get_model_price, train_model
from pckgs.price_preprocess import PricePreprocess
from sklearn.model_selection import train_test_split
from pckgs.evaluator import Evaluator
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
coins = {}
coin_names = ['BTC_USD', 'ETH_USD', 'XRP_USD', 'LTC_USD', 'BCH_USD', 'XLM_USD', 'LINK_USD']
coin_dirs = []
for i in range(len(coin_names)): coin_dirs.append('../Price/datasets/coinbase_hour_candles/' + coin_names[i] + '.feather')
coins = {coin_names[i]:coin_dirs[i] for i in range(len(coin_names))}
coins

{'BTC_USD': '../Price/datasets/coinbase_hour_candles/BTC_USD.feather',
 'ETH_USD': '../Price/datasets/coinbase_hour_candles/ETH_USD.feather',
 'XRP_USD': '../Price/datasets/coinbase_hour_candles/XRP_USD.feather',
 'LTC_USD': '../Price/datasets/coinbase_hour_candles/LTC_USD.feather',
 'BCH_USD': '../Price/datasets/coinbase_hour_candles/BCH_USD.feather',
 'XLM_USD': '../Price/datasets/coinbase_hour_candles/XLM_USD.feather',
 'LINK_USD': '../Price/datasets/coinbase_hour_candles/LINK_USD.feather'}

In [90]:
candle = pd.read_feather(coins['BTC_USD'])
candle.set_index('time', inplace=True)
candle

,open,high,low,close,volume
time,,,,,
2015-01-01 00:00:00+00:00,344.0,344.0,344.0,344.0,0.000000e+00
2015-01-01 01:00:00+00:00,344.0,344.0,344.0,344.0,0.000000e+00
2015-01-01 02:00:00+00:00,344.0,344.0,344.0,344.0,0.000000e+00
2015-01-01 03:00:00+00:00,344.0,344.0,344.0,344.0,0.000000e+00
2015-01-01 04:00:00+00:00,344.0,344.0,344.0,344.0,0.000000e+00
...,...,...,...,...,...
2019-12-30 20:00:00+00:00,7220.0,7228.0,7200.0,7214.0,2.046794e+06
2019-12-30 21:00:00+00:00,7212.0,7258.0,7210.0,7244.0,3.279451e+06
2019-12-30 22:00:00+00:00,7244.0,7258.0,7232.0,7242.0,1.443231e+06


In [91]:
lag = 21
threshold = 0.0025
pp = PricePreprocess(lag, threshold)
positions = pp.preprocess(candle)
positions


 Value of observations: 

up      16404
down    15187
same    12189
Name: pChange, dtype: int64


,pChange_scaled_t-1,pChange_scaled_t-2,pChange_scaled_t-3,pChange_scaled_t-4,pChange_scaled_t-5,pChange_scaled_t-6,pChange_scaled_t-7,pChange_scaled_t-8,pChange_scaled_t-9,pChange_scaled_t-10,...,pChange_scaled_t-14,pChange_scaled_t-15,pChange_scaled_t-16,pChange_scaled_t-17,pChange_scaled_t-18,pChange_scaled_t-19,pChange_scaled_t-20,down,same,up
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 21:00:00+00:00,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,0,1,0
2015-01-01 22:00:00+00:00,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,0,1,0
2015-01-01 23:00:00+00:00,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,0,1,0
2015-01-02 00:00:00+00:00,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,0,1,0
2015-01-02 01:00:00+00:00,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30 20:00:00+00:00,-0.047779,-0.044850,-0.046021,-0.058229,-0.050073,-0.041945,-0.046592,-0.043107,-0.050645,-0.055803,...,-0.044275,-0.037344,-0.048885,-0.041965,-0.043118,-0.059762,-0.046570,1,0,0
2019-12-30 21:00:00+00:00,-0.047197,-0.047779,-0.044850,-0.046021,-0.058229,-0.050073,-0.041945,-0.046592,-0.043107,-0.050645,...,-0.046001,-0.044275,-0.037344,-0.048885,-0.041965,-0.043118,-0.059762,0,0,1
2019-12-30 22:00:00+00:00,-0.036043,-0.047197,-0.047779,-0.044850,-0.046021,-0.058229,-0.050073,-0.041945,-0.046592,-0.043107,...,-0.044850,-0.046001,-0.044275,-0.037344,-0.048885,-0.041965,-0.043118,1,0,0


In [92]:
y = positions.loc[:, ['down', 'same', 'up']]
x = positions.drop(['down', 'same', 'up'], axis=1)

In [93]:
#same up until now!!!!!!!!!!!!!!
sentiment = pd.read_csv('../Text/datasets/headline_sentiment_mean.csv', index_col='date', parse_dates=['date'])

sentiment_score = timeseries_to_supervised2(sentiment, lag=21)
sentiment_score.dropna(inplace=True)
sentiment_score.drop('sentiment_score_t', axis=1, inplace=True)
x2 = sentiment_score
x2

,sentiment_score_t-1,sentiment_score_t-2,sentiment_score_t-3,sentiment_score_t-4,sentiment_score_t-5,sentiment_score_t-6,sentiment_score_t-7,sentiment_score_t-8,sentiment_score_t-9,sentiment_score_t-10,sentiment_score_t-11,sentiment_score_t-12,sentiment_score_t-13,sentiment_score_t-14,sentiment_score_t-15,sentiment_score_t-16,sentiment_score_t-17,sentiment_score_t-18,sentiment_score_t-19,sentiment_score_t-20
date,,,,,,,,,,,,,,,,,,,,
2015-01-21,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,0.142415,-0.061513,-0.032284,-0.096522,-0.167775,-0.114472,0.025266,-0.029244,0.057121,-0.168835
2015-01-22,0.116490,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,0.142415,-0.061513,-0.032284,-0.096522,-0.167775,-0.114472,0.025266,-0.029244,0.057121
2015-01-23,0.058076,0.116490,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,0.142415,-0.061513,-0.032284,-0.096522,-0.167775,-0.114472,0.025266,-0.029244
2015-01-24,0.102084,0.058076,0.116490,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,0.142415,-0.061513,-0.032284,-0.096522,-0.167775,-0.114472,0.025266
2015-01-25,-0.018787,0.102084,0.058076,0.116490,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,0.142415,-0.061513,-0.032284,-0.096522,-0.167775,-0.114472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,-0.173105,-0.035868,-0.010401,-0.017671,-0.030418,0.044368,0.086441,-0.008117,-0.113960,-0.136853,-0.130920,-0.076284,-0.048364,-0.027700,-0.002369,-0.024807,-0.113208,-0.031633,0.058666,0.025085
2019-12-28,-0.084293,-0.173105,-0.035868,-0.010401,-0.017671,-0.030418,0.044368,0.086441,-0.008117,-0.113960,-0.136853,-0.130920,-0.076284,-0.048364,-0.027700,-0.002369,-0.024807,-0.113208,-0.031633,0.058666
2019-12-29,0.030249,-0.084293,-0.173105,-0.035868,-0.010401,-0.017671,-0.030418,0.044368,0.086441,-0.008117,-0.113960,-0.136853,-0.130920,-0.076284,-0.048364,-0.027700,-0.002369,-0.024807,-0.113208,-0.031633


In [94]:
x = x2.merge(x, left_index=True, right_on=pd.to_datetime(x.index.strftime('%Y-%m-%d')), how='right').dropna()
x.drop(columns='key_0', inplace=True)
y = y.reindex(x.index)
x

,sentiment_score_t-1,sentiment_score_t-2,sentiment_score_t-3,sentiment_score_t-4,sentiment_score_t-5,sentiment_score_t-6,sentiment_score_t-7,sentiment_score_t-8,sentiment_score_t-9,sentiment_score_t-10,...,pChange_scaled_t-11,pChange_scaled_t-12,pChange_scaled_t-13,pChange_scaled_t-14,pChange_scaled_t-15,pChange_scaled_t-16,pChange_scaled_t-17,pChange_scaled_t-18,pChange_scaled_t-19,pChange_scaled_t-20
time,,,,,,,,,,,,,,,,,,,,,
2015-01-21 00:00:00+00:00,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850
2015-01-21 01:00:00+00:00,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850
2015-01-21 02:00:00+00:00,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850
2015-01-21 03:00:00+00:00,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850
2015-01-21 04:00:00+00:00,-0.129169,0.164252,0.082091,0.067433,0.038339,0.194653,0.028789,-0.067296,-0.052945,0.114822,...,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850,-0.044850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30 20:00:00+00:00,0.088101,0.030249,-0.084293,-0.173105,-0.035868,-0.010401,-0.017671,-0.030418,0.044368,0.086441,...,-0.048304,-0.044850,-0.046001,-0.044275,-0.037344,-0.048885,-0.041965,-0.043118,-0.059762,-0.046570
2019-12-30 21:00:00+00:00,0.088101,0.030249,-0.084293,-0.173105,-0.035868,-0.010401,-0.017671,-0.030418,0.044368,0.086441,...,-0.055803,-0.048304,-0.044850,-0.046001,-0.044275,-0.037344,-0.048885,-0.041965,-0.043118,-0.059762
2019-12-30 22:00:00+00:00,0.088101,0.030249,-0.084293,-0.173105,-0.035868,-0.010401,-0.017671,-0.030418,0.044368,0.086441,...,-0.050645,-0.055803,-0.048304,-0.044850,-0.046001,-0.044275,-0.037344,-0.048885,-0.041965,-0.043118


In [95]:
x = x.values.reshape((len(x), int(len(x.columns) / 2), 2), order='F')
print(x.shape)
print(y.shape)

(43321, 20, 2)
(43321, 3)


In [96]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

y_index = y_test.index
x_index = y_train.index